In [1]:
from pyspark import SparkContext                                                                                        
from pyspark.sql import SparkSession                                                                                    
from pyspark.streaming import StreamingContext                                                                          
from pyspark.streaming.kafka import KafkaUtils    
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import pandas as pd
import json
import time
import yaml

In [2]:
# ss = SparkSession.Builder() \
#      .appName("SparkStreamingKafka") \
#      .master("spark://streaming-spark-master:7077") \
#      .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.4.1.jar") \
#      .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/") \
#      .getOrCreate()
ss = SparkSession.Builder() \
     .appName("SparkSpeedStreamingKafka") \
     .master("spark://speed-processing-spark-master:7077") \
     .config("spark.jars", "./spark-sql-kafka-0-10_2.11-2.4.1.jar,./kafka-clients-2.0.0.jar") \
     .getOrCreate()


In [3]:
line = ss \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-broker-1:9093,kafka-broker-2:9093,kafka-broker-3:9093") \
  .option("subscribe", "trips") \
  .load()
# line.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
# Split the lines into words
# words = lines.select(
#    explode(
#        split(lines.value, " ")
#    ).alias("word")
# )

s = line.select("value")

# # Generate running word count
# wordCounts = words.groupBy("word").count()

In [ ]:
query = s \
    .writeStream \
    .trigger(continuous='5 seconds') \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker-1:9093,kafka-broker-2:9093,kafka-broker-3:9093") \
    .option("topic", "real-time-statistic") \
    .option("checkpointLocation", "/tmp/checkpoint") \
    .start()

query.awaitTermination()